In [1]:
from pathlib import Path

import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

from astropy import units as u
from astropy import table

import pyvo as vo

from tqdm.notebook import tqdm

from multiprocess import Pool

from dill import dump, load

In [2]:
import warnings
from astroquery.exceptions import NoResultsWarning
warnings.simplefilter('ignore', NoResultsWarning)
warnings.simplefilter('ignore', u.UnitsWarning) # comes up in the parsing of some heasarc tables but is just a "discouraged by FITS standard" issue

In [3]:
import SAGA
saga = SAGA.QuickStart('/Users/etollerud/Dropbox/', '/../')

SAGA `shared_dir` set to /Users/etollerud/Dropbox/
SAGA `local_dir`  set to /../


In [4]:
hosts = saga.host_catalog.load()

# First match HEASARC data

In [5]:
from astroquery.heasarc import Heasarc
heasarc = Heasarc()

CONCURRENCY_QRYS = 10

In [6]:
missions = heasarc.query_mission_list()
missions

Mission,Table,Table Description
str16,str12,str80
AGILE,agilecat,First AGILE Catalog of High-Confidence Gamma-Ray Sources
AGILE,agileupvar,Updated AGILE Catalog of Bright Gamma-Ray Sources and Variability
AKARI,akaribsc,"AKARI/FIS All-Sky Survey Bright Source Catalog, Version 1"
AKARI,akaripsc,"AKARI/IRC All-Sky Survey Point Source Catalog, Version 1"
ANS,ansuvpscat,ANS Ultraviolet Photometry Catalog of Point Sources
ARIEL V,ariel3a,3rd Ariel-V SSI Catalog
...,...,...
XMM-NEWTON,xmmstack,XMM-Newton Serendipitous Source Catalog from Stacked Observations (4XMM-DR13s)
XMM-NEWTON,xmmstackob,XMM-Newton Serendipitous Source Catalog from Stacked Observations: Obs. Data


In [7]:
list(np.unique(missions['Mission']))

['AGILE',
 'AKARI',
 'ANS',
 'ARIEL V',
 'ASCA',
 'BBXRT',
 'BEPPOSAX',
 'CALET',
 'CGRO',
 'CHANDRA',
 'COBE',
 'COPERNICUS',
 'COROT',
 'COS B',
 'EINSTEIN',
 'EUVE',
 'EXOSAT',
 'FAUST',
 'FERMI',
 'FUSE',
 'GALAXY CATALOG',
 'GALEX',
 'GAMMA-RAY BURSTS',
 'GENERAL CATALOG',
 'GINGA',
 'GROUND-BASED',
 'HALOSAT',
 'HEAO1',
 'HERSCHEL',
 'HETE-2',
 'HITOMI',
 'HST',
 'ICECUBE',
 'INTEGRAL',
 'IRAS',
 'ISO',
 'IUE',
 'IXPE',
 'KVANT',
 'LPF',
 'MASTER CATALOG',
 'MAXI',
 'MSX',
 'NEBULA CATALOG',
 'NICER',
 'NUSTAR',
 'OSO8',
 'PLANCK',
 'RADIO CATALOG',
 'RHESSI',
 'ROSAT',
 'RXTE',
 'SAS-2',
 'SAS-3',
 'SDSS',
 'SPITZER',
 'SRG-EROSITA',
 'STAR CATALOG',
 'SUZAKU',
 'SWIFT',
 'TD1',
 'UHURU',
 'UIT',
 'VELA 5B',
 'VERITAS',
 'WISE',
 'WMAP',
 'XMM-NEWTON']

In [8]:
xray_mission_tables = []

In [9]:
xray_mission_tables.append(missions[missions['Mission']=='SRG-EROSITA'])
xray_mission_tables[-1]

Mission,Table,Table Description
str16,str12,str80
SRG-EROSITA,efedshard,eROSITA eFEDS Hard X-Ray (2.3-5.0 keV) Source Catalog
SRG-EROSITA,efedsmain,eROSITA eFEDS Main X-Ray (0.2-2.3 keV) Source Catalog
SRG-EROSITA,erosmaster,eROSITA Observations Master Catalog
SRG-EROSITA,etachahard,Eta Chamaeleontis eROSITA EDR Field X-Ray (2.3-5.0 keV) Source Catalog
SRG-EROSITA,etachamain,Eta Chamaeleontis eROSITA EDR Field X-Ray (0.2-2.3 keV) Source Catalog


In [10]:
xray_mission_tables.append(missions[missions['Mission']=='XMM-NEWTON'])
xray_mission_tables[-1]

Mission,Table,Table Description
str16,str12,str80
XMM-NEWTON,agnsdssxm2,Sloan Digital Sky Survey/XMM-Newton Type1 AGN X-Ray and Radio Properties Catalog
XMM-NEWTON,agnsdssxmm,Sloan Digital Sky Survey/XMM-Newton AGN Spectral Properties Catalog
XMM-NEWTON,alfperxmm,Alpha Per Open Cluster XMM-Newton X-Ray Source Catalog
XMM-NEWTON,axis,AXIS XMM-Newton Source Catalog
XMM-NEWTON,caixa,Catalog of AGN in the XMM-Newton Archive
XMM-NEWTON,carinaxmm,Carina OB1 Association XMM-Newton X-Ray Point Source Catalog
...,...,...
XMM-NEWTON,xmmstack,XMM-Newton Serendipitous Source Catalog from Stacked Observations (4XMM-DR13s)
XMM-NEWTON,xmmstackob,XMM-Newton Serendipitous Source Catalog from Stacked Observations: Obs. Data


In [11]:
xray_mission_tables.append(missions[missions['Mission']=='ROSAT'])
xray_mission_tables[-1]

Mission,Table,Table Description
str16,str12,str80
ROSAT,bhrfscid,Byurakan/Hamburg/ROSAT Catalog of Optical IDs
ROSAT,bmwhricat,Brera Multi-scale Wavelet ROSAT HRI Source Catalog
ROSAT,chasfrxray,ROSAT All-Sky Survey: Chamaeleon Star Forming Region Study
ROSAT,cmar1xray,ROSAT PSPC Catalog of Canis Major R1 X-Ray Sources
ROSAT,dxrbs,ROSAT Deep X-Ray Radio Blazar Survey Catalog
ROSAT,etgalxray,Early-Type Galaxies X-Ray Luminosities Catalog
...,...,...
ROSAT,smcrosxry2,ROSAT PSPC Catalog of SMC X-Ray Sources (Haberl et al.)
ROSAT,spidersros,SDSS-IV/SPIDERS ROSAT-Based X-Ray Point Source Spectroscopic Catalog


In [12]:
xray_mission_tables.append(missions[missions['Mission']=='CHANDRA'])
xray_mission_tables[-1]

Mission,Table,Table Description
str16,str12,str80
CHANDRA,acceptcat,Archive of Chandra Cluster Entropy Profile Tables (ACCEPT) Catalog
CHANDRA,aegisx,AEGIS-X Chandra Extended Groth Strip X-Ray Point Source Catalog
CHANDRA,aegisxdcxo,AEGIS-X Deep Survey Chandra X-Ray Point Source Catalog
CHANDRA,aknepdfcxo,Akari North Ecliptic Pole Deep Field Chandra X-Ray Point Source Catalog
CHANDRA,arcquincxo,Arches and Quintuplet Clusters Chandra X-Ray Point Source Catalog
CHANDRA,bmwchancat,Brera Multi-scale Wavelet Chandra Source Catalog
...,...,...
CHANDRA,vlulxcat,Extremely Luminous X-Ray Source Candidates Catalog
CHANDRA,w40sfrcxo,W 40 Star-Forming Region Chandra X-Ray Point Source Catalog


In [13]:
xray_mission_all = table.vstack(xray_mission_tables)
xray_mission_masters = xray_mission_all[np.array(['master' in row['Table'] for row in xray_mission_all])]
xray_mission_masters

Mission,Table,Table Description
str16,str12,str80
SRG-EROSITA,erosmaster,eROSITA Observations Master Catalog
XMM-NEWTON,xmmmaster,XMM-Newton Master Log & Public Archive
ROSAT,rassmaster,ROSAT All-Sky Survey Archival Data
ROSAT,rosmaster,ROSAT Archival Data
CHANDRA,chanmaster,Chandra Observations


In [14]:
heasarc.query_region(hosts[35]['coord'], mission='chanmaster', radius=10.*u.degree)

OBSID,STATUS,NAME,RA,DEC,TIME,DETECTOR,GRATING,EXPOSURE,TYPE,PI,PUBLIC_DATE,SEARCH_OFFSET_
,,,deg,deg,mjd,,,s,,,mjd,
int32,bytes20,bytes50,float64,float64,bytes16,bytes6,bytes4,float64,bytes3,bytes40,int32,bytes39
16233,archived,NGC 1404,54.76700,-35.59200,56970.5976273148,ACIS-S,NONE,100060,GO,Kraft,57480,"565.847 (65.1095870053,-31.7245872504)\n"
16304,archived,MCS J0416.1-2403,64.03750,-24.06783,56616.8403356482,ACIS-I,NONE,99080,GTO,Murray,56983,"462.900 (65.1095870053,-31.7245872504)\n"
16234,archived,NGC 1404 tail,54.84300,-35.64500,56960.0303819444,ACIS-S,NONE,92060,GO,Kraft,57326,"563.563 (65.1095870053,-31.7245872504)\n"
3577,archived,RCS0439.6-2905,69.90833,-29.08200,52745.966712963,ACIS-S,NONE,77170,GTO,Garmire,53117,"294.578 (65.1095870053,-31.7245872504)\n"
16523,archived,MACSJ0416.1-2403,64.03583,-24.07794,57008.8562615741,ACIS-I,NONE,72070,GO,Jones,57010,"462.309 (65.1095870053,-31.7245872504)\n"
7191,archived,ClG J0405-4100,61.35125,-41.00417,53874.0065162037,ACIS-I,NONE,70130,GO,Vikhlinin,54241,"585.473 (65.1095870053,-31.7245872504)\n"
...,...,...,...,...,...,...,...,...,...,...,...,...
4458,archived,EARTH,75.10000,-36.45000,53063.2086689815,HRC-I,NONE,1250,GO,Elsner,53444,"571.231 (65.1095870053,-31.7245872504)\n"


In [15]:
heasarc.query_region(hosts[35]['coord'], mission='rosmaster', radius=10.*u.degree)

SEQ_ID,INSTRUMENT,EXPOSURE,NAME,RA,DEC,START_TIME,END_TIME,SEARCH_OFFSET_
,,s,,deg,deg,mjd,mjd,
bytes16,bytes6,int32,bytes20,float64,float64,bytes16,bytes16,bytes39
RH600831A01,HRI,88557,NGC 1399,54.6300,-35.4500,50271.6172685185,50321.6630208333,"569.094 (65.1095870053,-31.7245872504)\n"
RH600831N00,HRI,73509,NGC 1399,54.6300,-35.4500,50086.5952083333,50136.41,"569.094 (65.1095870053,-31.7245872504)\n"
RH600940N00,HRI,63877,NGC 1404,54.7200,-35.5900,50475.3758680556,50482.6659606481,"567.942 (65.1095870053,-31.7245872504)\n"
RP600043N00,PSPCB,53511,N1399,54.6200,-35.4500,48483.1505092593,48484.0520138889,"569.553 (65.1095870053,-31.7245872504)\n"
RH600676N00,HRI,42601,NGC 1380,54.1100,-34.9800,49923.6369097222,49925.3159027778,"584.459 (65.1095870053,-31.7245872504)\n"
RH600839N00,HRI,38362,ESO359-29,63.2100,-33.0000,50321.2355555556,50322.3530671296,"122.977 (65.1095870053,-31.7245872504)\n"
...,...,...,...,...,...,...,...,...
RP190924N00,PSPCB,0,,71.3179,-40.3460,50507.7731828704,50507.7749074074,"598.201 (65.1095870053,-31.7245872504)\n"


In [16]:
heasarc.query_region(hosts[35]['coord'], mission='rassmaster', radius=10.*u.degree)

SEQ_ID,INSTRUMENT,EXPOSURE,RA,DEC,TITLE,PUBLIC_DATE,SEARCH_OFFSET_
,,s,deg,deg,,mjd,
bytes16,bytes6,int32,float64,float64,bytes20,int32,bytes39
RS932410N00,PSPC,791,65.76692,-39.37500,RASS 3/24/10,51615,"460.141 (65.1095870053,-31.7245872504)\n"
RS932311N00,PSPC,718,68.72521,-33.75000,RASS 3/23/11,51615,"219.216 (65.1095870053,-31.7245872504)\n"
RS932310N00,PSPC,717,62.17942,-33.75000,RASS 3/23/10,51615,"191.394 (65.1095870053,-31.7245872504)\n"
RS932409N00,PSPC,678,58.84583,-39.37500,RASS 3/24/09,51615,"551.197 (65.1095870053,-31.7245872504)\n"
RS932312N00,PSPC,659,75.27100,-33.75000,RASS 3/23/12,51615,"526.794 (65.1095870053,-31.7245872504)\n"
RS932211N00,PSPC,629,65.17096,-28.12500,RASS 3/22/11,51615,"215.999 (65.1095870053,-31.7245872504)\n"
RS932212N00,PSPC,629,71.37917,-28.12500,RASS 3/22/12,51615,"390.948 (65.1095870053,-31.7245872504)\n"
RS932210N00,PSPC,628,58.96279,-28.12500,RASS 3/22/10,51615,"385.646 (65.1095870053,-31.7245872504)\n"


In [17]:
heasarc.query_region(hosts[35]['coord'], mission='xmmmaster', radius=10.*u.degree)

OBSID,STATUS,NAME,RA,DEC,TIME,DURATION,PI_LNAME,PI_FNAME,PUBLIC_DATE,DATA_IN_HEASARC,SEARCH_OFFSET_
,,,deg,deg,mjd,s,,,mjd,,
bytes10,bytes9,bytes24,float64,float64,bytes16,int32,bytes20,bytes20,int32,bytes1,bytes39
0804790501,archived,WASP-79,66.37090,-30.60044,57952.221875,12000,Sanz-Forcada,Jorge,58331,Y,"93.503 (65.1095870053,-31.7245872504)\n"
0405730401,archived,SN 2003bg,62.74758,-31.41397,54129.8308912037,21910,Filippenko,Alex,54530,Y,"122.174 (65.1095870053,-31.7245872504)\n"
0881990201,archived,SN2003bg,62.74758,-31.41394,59601.4691898148,20400,Stroh,Michael,59978,Y,"122.174 (65.1095870053,-31.7245872504)\n"
0765000901,archived,A3223,62.06750,-30.89444,57415.7816782407,11000,Reiprich,Thomas,57798,Y,"163.699 (65.1095870053,-31.7245872504)\n"
0301330401,archived,F04103-2838,63.08125,-28.50678,53779.3893865741,22855,Wilson,Andrew,54165,Y,"219.890 (65.1095870053,-31.7245872504)\n"
0742500301,archived,DH Eri,65.91791,-27.83027,56907.2393402778,15000,Sahai,Raghvendra,57283,Y,"237.418 (65.1095870053,-31.7245872504)\n"
...,...,...,...,...,...,...,...,...,...,...,...
0550931001,archived,Fornax offset J,54.41201,-35.75500,54642.7047569444,21820,Matsushita,Kyoko,55043,Y,"585.520 (65.1095870053,-31.7245872504)\n"


In [18]:
heasarc.query_region(hosts[35]['coord'], mission='erosmaster', radius=10.*u.degree)

<Table length=0>
<No columns>

Alright, this seems to be working as expected for everything except eROSITA.

So what we want to do is for each host, query the relevant "master" catalog with an area that's about matched to whatever the main detector's FOV is (generously) to find which have any observations.  Only from those do we search for actual sources.

For eROSITA it turns out it's a totally separate source, so we skip that and use the vo cone search directly from eROSITA instead of HEASARC.  Similarly, for the ROSAT rassmaster, the all-sky-survey is supposed to be, well, an all-sky survey so we skip it.

In [19]:
mission_to_rad = {'xmmmaster': 15*u.arcmin, # " telescope's field of view (FOV) of 30 arcmin and in th"
                  'chanmaster':8*u.arcmin, # this is ~ half ASIC's FOV
                  'rosmaster':2.5*u.deg,  #"5 degree diameter"
                  'rassmaster':2.5*u.deg,  #"5 degree diameter"
                }

In [20]:
host_results = []
    
for host in tqdm(hosts[:2]):
    reses = {}
    for tabname, rad in mission_to_rad.items():
        reses[tabname] = heasarc.query_region(host['coord'], mission=tabname, radius=rad)
    host_results.append(reses)
            


  0%|          | 0/2 [00:00<?, ?it/s]

Better parallel:

In [21]:
pres = Path('heasarc_master_results.pickle')

results = []
if pres.exists():
    print(f'{pres} already exists, not overwriting, loading')
    with pres.open('rb') as f:
        results.extend(load(f))
else:
    with Pool(CONCURRENCY_QRYS) as p:
        reses = []
        for host in hosts:
            for tabname, rad in mission_to_rad.items():
                ares = p.apply_async(heasarc.query_region, (host['coord'],), kwds={'mission':tabname, 'radius':rad})
                reses.append((host['HOSTID'], tabname, ares))
        t = tqdm(total=len(reses))
        while t.n < t.total:
            nrdy = sum([r[2].ready() for r in reses])
            if nrdy > t.n:
                t.update(nrdy - t.n)
    
        for r in reses:
            results.append((r[0], r[1], r[2].get()))

    print(f'writing to {pres}')
    with pres.open('bw') as f:
        dump(results, f)

heasarc_master_results.pickle already exists, not overwriting, loading


In [22]:
for tabnm in mission_to_rad:
    hosts['in_'+tabnm] = [False for _ in range(len(hosts))] 

for hnm, tabnm, tab in results:
    if len(tab)>0:
        hosts['in_'+tabnm][hosts['HOSTID']==hnm] = True
hosts

HOSTID,PGC,SAGA_NAME,COMMON_NAME,NSAID,NSA1ID,RA,DEC,GLON,GLAT,V_HELIO,V_VIRGO,Z_COSMO,Z_HELIO,DIST,DISTMOD,K_RAW,K_TC,K_ABS,M_HALO,REMOVED_BY_HAND,SGA_ID,MORPHTYPE,LOG_MHI,BRIGHTEST_K_R1,BRIGHTEST_K_R2,BRIGHTEST_K_BG_R1,BRIGHTEST_K_BG_R2,BRIGHTEST_STAR_R1,STAR_DENSITY_R1,BRIGHTEST_STAR_R2,STAR_DENSITY_R2,COVERAGE_DECALS_DR5,COVERAGE_DECALS_DR6,COVERAGE_DECALS_DR7,COVERAGE_DECALS_DR8,COVERAGE_DECALS_DR9,COVERAGE_DES_DR1,COVERAGE_SDSS,HOST_SCORE,HAS_IMAGE,NEAREST_MW,NEAREST_MW_DIST,NEAREST_MW_SEP,NEARBY_MW_COUNT_1,NEARBY_MW_COUNT_1.5,NEARBY_MW_COUNT_2,NEAREST_MASSIVE,NEAREST_MASSIVE_DIST,NEAREST_MASSIVE_SEP,NEARBY_MASSIVE_COUNT_1,NEARBY_MASSIVE_COUNT_1.5,NEARBY_MASSIVE_COUNT_2,coord,in_xmmmaster,in_chanmaster,in_rosmaster,in_rassmaster
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"deg,deg",,,,
str10,int64,str13,str22,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,str9,float64,float64,int64,int64,int64,str9,float64,float64,int64,int64,int64,SkyCoord,bool,bool,bool,bool
nsa126115,279,Bandamanna,NGC7817,126115,613769,0.995382,20.7523559,108.2284018,-40.7605243,2309.4,2379.7,0.007509265978,0.007703329215,32.35243745,32.54953503,8.4,8.413236622,-24.13629841,12.22914145,0,1359702,Sbc,9.564883996,99.0,99.0,99.0,10.04360438,7.6193,0.0,8.1267,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,4,2,nsa153017,1.796472819,1.065293456,0,0,1,nsa153017,1.796472819,1.065293456,0,0,1,"0.995382,20.7523559",False,False,True,False
pgc1952,1952,,ESO079-003,-1,-1,8.008776,-64.2533874,306.4105111,-52.7435281,2623.3,2355.9,0.007833922449,0.008750386909,33.75981049,32.642,9.01,9.018797428,-23.62320257,12.04514145,0,509348,SBb,9.479355042,11.808,99.0,11.968,99.0,9.4397,0.0,6.5878,0.4092455203,0.0,0.0,0.0,1.0,1.0,1.0,0.0,4,2,pgc3743,3.984681028,3.532546535,0,0,0,pgc3743,3.984681028,3.532546535,0,0,0,"8.008776,-64.2533874",False,False,True,False
pgc2052,2052,,NGC0150,-1,-1,8.564589,-27.8036183,21.8951315,-86.130036,1577.8,1407.1,0.004890905952,0.005262974294,21.02809837,31.614,8.506,8.516911926,-23.09708807,11.56114145,0,1165798,SBbc,nan,99.0,11.661,10.49967197,9.866801199,8.0514,0.0,7.1699,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2,2,pgc3089,1.930768513,5.26053089,0,0,1,pgc3089,1.930768513,5.26053089,0,0,1,"8.564589,-27.8036183",False,False,True,True
nsa127226,2081,,NGC0157,127226,614858,8.694936,-8.3964957,110.2762933,-70.8578303,1655.1,1566.9,0.006427784722,0.00552081934,27.66941645,32.21,7.665,7.679767546,-24.53023245,12.20914145,0,895869,SABb,10.09376372,99.0,99.0,10.34780753,9.609372536,8.1775,0.0,8.2894,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,4,2,nsa128381,5.463821353,7.379299229,0,0,0,nsa128381,5.463821353,7.379299229,0,0,0,"8.694936,-8.3964957",False,True,True,False
pgc2778,2778,,NGC0254,-1,-1,11.864958,-31.4216944,314.1143294,-85.6198155,1626.1,1434.8,0.004947318959,0.005424085752,21.27159227,31.639,8.72,8.730921303,-22.9080787,11.87914145,0,242698,S0-a,nan,99.0,7.998540595,11.52414343,9.998289411,7.2489,0.0,6.969,0.1624415358,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2,2,pgc3089,0.6050530296,1.140973174,1,1,2,pgc3089,0.6050530296,1.140973174,1,1,2,"11.864958,-31.4216944",False,False,True,False
pgc3089,3089,,NGC0289,-1,-1,13.176354,-31.2058056,299.1263539,-85.9130198,1629.6,1436.3,0.004846237289,0.005435760495,20.83531161,31.594,7.99,7.998540595,-23.59545941,12.10414145,0,1138552,SBbc,nan,99.0,8.730921303,9.998289411,10.41550149,9.31,0.0,7.2489,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2,2,pgc2052,1.930768513,5.26053089,0,0,1,pgc2052,1.930768513,5.26053089,0,0,1,"13.176354,-31.2058056",False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pgc72060,72060,,NGC7727,-1,-1,354

In [23]:
np.sum(hosts['in_chanmaster'])/len(hosts)

0.2

In [24]:
np.sum(hosts['in_xmmmaster'])/len(hosts)

0.12666666666666668

In [25]:
np.sum(hosts['in_rosmaster'])/len(hosts)

0.8

In [26]:
np.sum(hosts['in_rassmaster'])/len(hosts)

0.5566666666666666

Now check the relevant source catalogs for anything that's in a footprint

In [27]:
chandra_tables = xray_mission_all[np.array([row['Mission']=='CHANDRA' and 'master' not in row['Table'] for row in xray_mission_all])]
chandra_tables

Mission,Table,Table Description
str16,str12,str80
CHANDRA,acceptcat,Archive of Chandra Cluster Entropy Profile Tables (ACCEPT) Catalog
CHANDRA,aegisx,AEGIS-X Chandra Extended Groth Strip X-Ray Point Source Catalog
CHANDRA,aegisxdcxo,AEGIS-X Deep Survey Chandra X-Ray Point Source Catalog
CHANDRA,aknepdfcxo,Akari North Ecliptic Pole Deep Field Chandra X-Ray Point Source Catalog
CHANDRA,arcquincxo,Arches and Quintuplet Clusters Chandra X-Ray Point Source Catalog
CHANDRA,bmwchancat,Brera Multi-scale Wavelet Chandra Source Catalog
...,...,...
CHANDRA,vlulxcat,Extremely Luminous X-Ray Source Candidates Catalog
CHANDRA,w40sfrcxo,W 40 Star-Forming Region Chandra X-Ray Point Source Catalog


Chandra has *arcsec* resolution, but I'm not confident I understand enough about extended source catalogs to know how to interpret that for something as sensitive as Chandra.  So we'll just choose 2 arcmin as a not-crazy scale to be conservative about

In [29]:
pres = Path('heasarc_chandra_results.pickle')

results = []
if pres.exists():
    print(f'{pres} already exists, not overwriting, loading')
    with pres.open('rb') as f:
        results.extend(load(f))
else:
    rad = 2*u.arcmin

    chunk_edges = np.linspace(0, len(hosts), 10)
    host_chunks = [hosts[int(i):int(j)] for i,j in zip(chunk_edges[:-1], chunk_edges[1:])]
    
    for hosts_chunk in tqdm(host_chunks):
        with Pool(CONCURRENCY_QRYS) as p:
            reses = []
            for host in hosts_chunk:
                for tabname in chandra_tables['Table']:
                    if host['in_chanmaster']:
                        ares = p.apply_async(heasarc.query_region, (host['coord'],), kwds={'mission':tabname, 'radius':rad})
                        reses.append((host['HOSTID'], tabname, ares))
                    else:
                        reses.append((host['HOSTID'], tabname, None))
            t = tqdm(total=len([r for r in reses if r[2] is not None]))
            while t.n < t.total:
                nrdy = sum([r[2].ready() for r in reses if r[2] is not None])
                if nrdy > t.n:
                    t.update(nrdy - t.n)
        
            for r in reses:
                results.append((r[0], r[1], None if r[2] is None else r[2].get()))
            
    print(f'writing to {pres}')
    with pres.open('bw') as f:
        dump(results, f)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/2400 [00:00<?, ?it/s]

KeyboardInterrupt: 

The above keeps freezing up.  Lets try just a few selected major catalogs instead

In [34]:
pres = Path('heasarc_chandra_results.pickle')

results = []
if pres.exists():
    print(f'{pres} already exists, not overwriting, loading')
    with pres.open('rb') as f:
        results.extend(load(f))
else:
    rad = 2*u.arcmin

    chunk_edges = np.linspace(0, len(hosts), 2)
    host_chunks = [hosts[int(i):int(j)] for i,j in zip(chunk_edges[:-1], chunk_edges[1:])]
    
    for hosts_chunk in tqdm(host_chunks):
        with Pool(CONCURRENCY_QRYS) as p:
            reses = []
            for host in hosts_chunk:
                for tabname in ['csc']:
                    if host['in_chanmaster']:
                        ares = p.apply_async(heasarc.query_region, (host['coord'],), kwds={'mission':tabname, 'radius':rad})
                        reses.append((host['HOSTID'], tabname, ares))
                    else:
                        reses.append((host['HOSTID'], tabname, None))
            t = tqdm(total=len([r for r in reses if r[2] is not None]))
            while t.n < t.total:
                nrdy = sum([r[2].ready() for r in reses if r[2] is not None])
                if nrdy > t.n:
                    t.update(nrdy - t.n)
        
            for r in reses:
                results.append((r[0], r[1], None if r[2] is None else r[2].get()))
            
    print(f'writing to {pres}')
    with pres.open('bw') as f:
        dump(results, f)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
hosts['in_chandra_source'] = [False]*len(hosts)

for hnm, tabnm, tab in results:
    if len(tab)>0:
        hosts['in_chandra_source'][hosts['HOSTID']==hnm] = True
hosts[hosts['in_chandra_source']

In [ ]:
hosts['in_chandra_source'] = [False]*len(hosts)

for hnm, tabnm, tab in results:
    if  'master' not in tabnm and len(tab)>0:
        hosts['in_chandra_source'][hosts['HOSTID']==hnm] = True
hosts[hosts['in_chandra_source']]

Xmm-newton claims ~arcsec as a resolution.  But take the same approach as Chandra for safety

In [83]:
xmmn_tables = xray_mission_all[np.array([row['Mission']=='XMM-NEWTON' and 'master' not in row['Table'] for row in xray_mission_all])]
xmmn_tables

Mission,Table,Table Description
str16,str12,str80
XMM-NEWTON,agnsdssxm2,Sloan Digital Sky Survey/XMM-Newton Type1 AGN X-Ray and Radio Properties Catalog
XMM-NEWTON,agnsdssxmm,Sloan Digital Sky Survey/XMM-Newton AGN Spectral Properties Catalog
XMM-NEWTON,alfperxmm,Alpha Per Open Cluster XMM-Newton X-Ray Source Catalog
XMM-NEWTON,axis,AXIS XMM-Newton Source Catalog
XMM-NEWTON,caixa,Catalog of AGN in the XMM-Newton Archive
XMM-NEWTON,carinaxmm,Carina OB1 Association XMM-Newton X-Ray Point Source Catalog
...,...,...
XMM-NEWTON,xmmstack,XMM-Newton Serendipitous Source Catalog from Stacked Observations (4XMM-DR13s)
XMM-NEWTON,xmmstackob,XMM-Newton Serendipitous Source Catalog from Stacked Observations: Obs. Data


In [84]:
pres = Path('heasarc_xmmnewton_results.pickle')

results = []
if pres.exists():
    print(f'{pres} already exists, not overwriting, loading')
    with pres.open('rb') as f:
        results.extend(load(f))
else:
    rad = 2*u.arcmin
    results = []
    
    with Pool(CONCURRENCY_QRYS) as p:
        reses = []
        for host in hosts:
            for tabname in xmmn_tables['Table']:
                if host['in_xmmmaster']:
                    ares = p.apply_async(heasarc.query_region, (host['coord'],), kwds={'mission':tabname, 'radius':rad})
                    reses.append((host['HOSTID'], tabname, ares))
                else:
                    reses.append((host['HOSTID'], tabname, None))
        t = tqdm(total=len([r for r in reses if r[2] is not None]))
        while t.n < t.total:
            nrdy = sum([r[2].ready() for r in reses if r[2] is not None])
            if nrdy > t.n:
                t.update(nrdy - t.n)
    
        for r in reses:
            results.append((r[0], r[1], None if r[2] is None else r[2].get()))
            
    print(f'writing to {pres}')
    with pres.open('bw') as f:
        dump(results, f)

  0%|          | 0/3914 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
hosts['in_xmmn_source'] = [False]*len(hosts)

for hnm, tabnm, tab in results:
    if  'master' not in tabnm and len(tab)>0:
        hosts['in_xmmn_source'][hosts['HOSTID']==hnm] = True
hosts[hosts['in_xmmn_source']]

ROSAT PSPC instrument claims 25" at 1 kev (https://heasarc.gsfc.nasa.gov/docs/rosat/pspc.html), so we will take 1 arcmin to be a match, since probably only Point Source are bright enough for rosat

In [ ]:
rosat_tables = xray_mission_all[np.array([row['Mission']=='ROSAT' and 'master' not in row['Table'] for row in xray_mission_all])]
rosat_tables

In [ ]:
pres = Path('heasarc_rosat_results.pickle')

results = []
if pres.exists():
    print(f'{pres} already exists, not overwriting, loading')
    with pres.open('rb') as f:
        results.extend(load(f))
else:
    rad = 2*u.arcmin
    results = []
    
    with Pool(CONCURRENCY_QRYS) as p:
        reses = []
        for host in hosts:
            for tabname in rosat_tables['Table']:
                if host['in_rosmaster'] | host['in_rassmaster']:
                    ares = p.apply_async(heasarc.query_region, (host['coord'],), kwds={'mission':tabname, 'radius':rad})
                    reses.append((host['HOSTID'], tabname, ares))
                else:
                    reses.append((host['HOSTID'], tabname, None))
        t = tqdm(total=len([r for r in reses if r[2] is not None]))
        while t.n < t.total:
            nrdy = sum([r[2].ready() for r in reses if r[2] is not None])
            if nrdy > t.n:
                t.update(nrdy - t.n)
    
        for r in reses:
            results.append((r[0], r[1], None if r[2] is None else r[2].get()))
            
    print(f'writing to {pres}')
    with pres.open('bw') as f:
        dump(results, f)

In [ ]:
hosts['in_rosat_source'] = [False]*len(hosts)

for hnm, tabnm, tab in results:
    if  'master' not in tabnm and len(tab)>0:
        hosts['in_rosat_source'][hosts['HOSTID']==hnm] = True
hosts[hosts['in_rosat_source']]